In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import math
np.random.seed(365)
from numpy import mean
import pickle
import warnings
warnings.filterwarnings("ignore")

import statsmodels.api as sm
from sklearn.metrics import mean_squared_error
from statsmodels.tools.eval_measures import rmse 
from statsmodels.tsa.api import ExponentialSmoothing, SimpleExpSmoothing, Holt
from statsmodels.tsa.arima_model import ARIMA
from statsmodels.tsa.stattools import adfuller
from statsmodels.tsa.seasonal import seasonal_decompose 
from pmdarima import auto_arima 
from statsmodels.tsa.statespace.sarimax import SARIMAX 

 


## Load the data:

In [4]:
queens_df = pickle.load(open('./data/final_cleaned_data/queens_cleaned_data.pkl','rb'))

### Splitting precovid and covid season:

In [5]:
#Queens
queens_precovid_df = queens_df[:'2020-03-21']
queens_covid_df = queens_df['2020-03-21':]

### Train Test Split:

In [6]:
#Queens
train_ = queens_df[:'2020-01-31']
test_ = queens_df['2020-02-01':'2020-02-29']

#### Set hour dataframe:

In [7]:
copied_df = train_.copy()
hour = pd.DataFrame(copied_df.index.hour)
hour

,pickup_time
0,0
1,1
2,2
3,3
4,4
...,...
5875,19
5876,20
5877,21
5878,22


In [8]:
copied_df['hour'] = copied_df.index.hour

#### Set day of the week (5 and 6 are Saturday and Sunday):

In [9]:
copied_df.index.dayofweek

Int64Index([5, 5, 5, 5, 5, 5, 5, 5, 5, 5,
            ...
            4, 4, 4, 4, 4, 4, 4, 4, 4, 4],
           dtype='int64', name='pickup_time', length=5880)

In [10]:
# assign week day and weekend
copied_df['day'] = copied_df.index.dayofweek

In [11]:
copied_df['day']

pickup_time
2019-06-01 00:00:00    5
2019-06-01 01:00:00    5
2019-06-01 02:00:00    5
2019-06-01 03:00:00    5
2019-06-01 04:00:00    5
                      ..
2020-01-31 19:00:00    4
2020-01-31 20:00:00    4
2020-01-31 21:00:00    4
2020-01-31 22:00:00    4
2020-01-31 23:00:00    4
Freq: H, Name: day, Length: 5880, dtype: int64

#### Dummy the data by using OneHotEncoder

In [12]:
from sklearn.preprocessing import OneHotEncoder
ohe = OneHotEncoder(drop='first')
output = ohe.fit_transform(copied_df[['hour', 'day']]).toarray() ## fit transform hour and day

In [13]:
output_df = pd.DataFrame(data=output, index=copied_df.index, columns=ohe.get_feature_names(['hour', 'day']))

In [14]:
output_df

,hour_1,hour_2,hour_3,hour_4,hour_5,hour_6,hour_7,hour_8,hour_9,hour_10,...,hour_20,hour_21,hour_22,hour_23,day_1,day_2,day_3,day_4,day_5,day_6
pickup_time,,,,,,,,,,,,,,,,,,,,,
2019-06-01 00:00:00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
2019-06-01 01:00:00,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
2019-06-01 02:00:00,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
2019-06-01 03:00:00,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
2019-06-01 04:00:00,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2020-01-31 19:00:00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
2020-01-31 20:00:00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
2020-01-31 21:00:00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0


#### Create SARIMAX Model: SEASONALITY should be 24 but it takes too long so I changed for this purpose:

In [ ]:
from statsmodels.tsa.statespace.sarimax import SARIMAX

order = (1,0,2)
seasonal_order = (1, 0, 2, 24)

model = SARIMAX(copied_df['count'], exog= output_df, order=order, freq='H')
sarimax_1 = model.fit()


In [ ]:
sarimax_1.summary()

#### Let's set up exogenous variables:

In [ ]:
exog2_df = pd.DataFrame(index=pd.date_range(start="2020-02-01", end="2020-02-29", freq='H'))

In [ ]:
exog2_df['hour'] = exog2_df.index.hour
exog2_df['day'] = exog2_df.index.dayofweek ## create day 

In [ ]:
exog2_df

In [ ]:
ohe_2 = ohe.transform(exog2_df[['hour', 'day']]).toarray()

In [ ]:
predictions = sarimax_1.predict(start='2020-02-01', end='2020-03-01', exog=ohe_2)
residuals = residuals[:'2020-02-28 23:00:00'] - predictions
residuals

In [ ]:
plt.figure(figsize=(16,6))
plt.plot(test_['count'])
plt.plot(predictions)
#plt.plot(train_['count'])
plt.title('SARIMAX ')
plt.legend(['Test Data', 'Predictions'])

## Forecasting: 


once picking final model after evaluation: ** REFIT** 

In [ ]:
# Refit last step
refit_ = queens_df[:'2020-02-29']
hour = pd.DataFrame(copied_df.index.hour)
refit_['hour'] = refit_.index.hour
# assign week day and weekend
refit_['day'] = refit_.index.dayofweek

In [ ]:
ohe_refit = ohe.fit_transform(refit_[['hour', 'day']]).toarray() ## fit transform hour and day

In [ ]:
exog_refit = pd.DataFrame(data=ohe_refit, index=refit_.index, columns=ohe.get_feature_names(['hour', 'day']))

In [ ]:
order = (1,0,2)
seasonal_order = (1, 0, 2, 8)

model_refit = SARIMAX(refit_['count'], exog= ohe_refit, order=order, freq='H')
sarimax_refit = model_refit.fit()


In [ ]:
exog_refit = pd.DataFrame(index=pd.date_range(start="2020-03-01", end="2020-04-01", freq='H'))
exog_refit['hour'] = exog_refit.index.hour
exog_refit['day'] = exog_refit.index.dayofweek ## create day 

In [ ]:
ohe_refit2 = ohe.transform(exog_refit[['hour', 'day']]).toarray()

In [ ]:
# predict
predictions_refit = sarimax_refit.predict(start='2020-03-01', end='2020-04-01', exog=ohe_refit2)
predictions_refit

In [ ]:
refit_

In [ ]:
plt.figure(figsize=(16,6))
plt.plot(refit_['count'])
plt.plot(predictions_refit)
#plt.plot(train_['count'])
plt.title('SARIMAX - Precovid')
plt.legend(['Test Data', 'Predictions'])